<a href="https://colab.research.google.com/github/prajjwalpandey19/AI_Content_Classifier/blob/main/Untitled0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os, json, zipfile, textwrap
from google.colab import files
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow.keras import layers, models, callbacks
import matplotlib.pyplot as plt

In [4]:
uploaded = files.upload()
csv_path = list(uploaded.keys())[0]
df = pd.read_csv('/content/drive/MyDrive/ibm sem 7/ai_human_content_detection_dataset.csv')

Saving ai_human_content_detection_dataset.csv to ai_human_content_detection_dataset.csv


In [9]:
# ============================================================
# 2️⃣ Detect text + label columns
# ============================================================
text_cols = [c for c in df.columns if df[c].dtype == "object"]
if text_cols:
    avg_len = {c: df[c].dropna().astype(str).map(len).mean() for c in text_cols}
    text_col = max(avg_len, key=avg_len.get)
else:
    text_col = df.columns[0]

label_col = None
for c in df.columns:
    if c == text_col: continue
    if 2 <= df[c].nunique() <= 10:
        label_col = c
        break
if label_col is None:
    label_col = df.columns[-1]

print(f"📝 Text column: {text_col}")
print(f"🏷️ Label column: {label_col}")

df = df[[text_col, label_col]].dropna().copy()
df[text_col] = df[text_col].astype(str)
le = LabelEncoder()
df["_label_enc"] = le.fit_transform(df[label_col])
num_classes = len(le.classes_)
print("Detected classes:", list(le.classes_))

train_df, val_df = train_test_split(df, test_size=0.15, stratify=df["_label_enc"], random_state=42)



📝 Text column: text_content
🏷️ Label column: content_type
Detected classes: ['academic_paper', 'article', 'blog_post', 'creative_writing', 'essay', 'news_article', 'product_review', 'social_media']


In [7]:
# ============================================================
# 3️⃣ Build Model (BiLSTM)
# ============================================================
MAX_TOKENS = 40000
SEQ_LEN = 300
EMBED_DIM = 128
BATCH = 64
EPOCHS = 40

vectorize = layers.TextVectorization(max_tokens=MAX_TOKENS, output_mode="int", output_sequence_length=SEQ_LEN)
vectorize.adapt(tf.data.Dataset.from_tensor_slices(train_df[text_col]).batch(BATCH))

inputs = tf.keras.Input(shape=(1,), dtype=tf.string)
x = vectorize(inputs)
x = layers.Embedding(MAX_TOKENS, EMBED_DIM, mask_zero=True)(x)
x = layers.Bidirectional(layers.LSTM(128, dropout=0.3, recurrent_dropout=0.2))(x)
x = layers.Dense(128, activation="relu")(x)
x = layers.Dropout(0.3)(x)
outputs = layers.Dense(num_classes, activation="softmax")(x)
model = models.Model(inputs, outputs)

model.compile(optimizer=tf.keras.optimizers.Adam(1e-4),
              loss="sparse_categorical_crossentropy", metrics=["accuracy"])
model.summary()



Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ text_vectorization  │ (None, 300)       │          0 │ input_layer[0][0] │
│ (TextVectorization) │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding           │ (None, 300, 128)  │  5,120,000 │ text_vectorizati… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ not_equal           │ (None, 300)       │          0 │ text_vectorizati… │
│ (NotEqual)          │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bidirectional       │ (None, 256)       │    263,168 │ embedding[0][0],  │
│ (Bidirectional)     │                   │            │ not_equal[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 128)       │     32,896 │ bidirectional[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128)       │          0 │ dense[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 8)         │      1,032 │ dropout[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 5,417,096 (20.66 MB)

 Trainable params: 5,417,096 (20.66 MB)

 Non-trainable params: 0 (0.00 B)

In [8]:
# ============================================================
# 4️⃣ Train Model
# ============================================================
def df_to_ds(df_, shuffle=True):
    ds = tf.data.Dataset.from_tensor_slices((df_[text_col].values, df_["_label_enc"].values))
    if shuffle: ds = ds.shuffle(len(df_), seed=42)
    return ds.batch(BATCH).prefetch(tf.data.AUTOTUNE)

train_ds = df_to_ds(train_df, True)
val_ds = df_to_ds(val_df, False)

cb_early = callbacks.EarlyStopping(monitor="val_loss", patience=5, restore_best_weights=True)
cb_reduce = callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3)

history = model.fit(train_ds, validation_data=val_ds, epochs=EPOCHS, callbacks=[cb_early, cb_reduce])


Epoch 1/40
19/19 ━━━━━━━━━━━━━━━━━━━━ 45s 2s/step - accuracy: 0.1325 - loss: 2.0794 - val_accuracy: 0.1553 - val_loss: 2.0781 - learning_rate: 1.0000e-04
Epoch 2/40
19/19 ━━━━━━━━━━━━━━━━━━━━ 47s 2s/step - accuracy: 0.1665 - loss: 2.0780 - val_accuracy: 0.1505 - val_loss: 2.0779 - learning_rate: 1.0000e-04
Epoch 3/40
19/19 ━━━━━━━━━━━━━━━━━━━━ 75s 2s/step - accuracy: 0.1646 - loss: 2.0772 - val_accuracy: 0.1456 - val_loss: 2.0777 - learning_rate: 1.0000e-04
Epoch 4/40
19/19 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.2100 - loss: 2.0754 - val_accuracy: 0.1650 - val_loss: 2.0775 - learning_rate: 1.0000e-04
Epoch 5/40
19/19 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.2219 - loss: 2.0742 - val_accuracy: 0.1553 - val_loss: 2.0773 - learning_rate: 1.0000e-04
Epoch 6/40
19/19 ━━━━━━━━━━━━━━━━━━━━ 36s 2s/step - accuracy: 0.2046 - loss: 2.0737 - val_accuracy: 0.1456 - val_loss: 2.0770 - learning_rate: 1.0000e-04
Epoch 7/40
19/19 ━━━━━━━━━━━━━━━━━━━━ 41s 2s/step - accuracy: 0.2048 - loss:

In [10]:
# ============================================================
# 5️⃣ Evaluate Model
# ============================================================
loss, acc = model.evaluate(val_ds)
print(f"✅ Validation accuracy: {acc:.4f}")

plt.figure(figsize=(8,4))
plt.plot(history.history["accuracy"], label="train acc")
plt.plot(history.history["val_accuracy"], label="val acc")
plt.title("Training Accuracy")
plt.legend()
plt.grid(True)
plt.show()


4/4 ━━━━━━━━━━━━━━━━━━━━ 1s 245ms/step - accuracy: 0.3053 - loss: 1.7895
✅ Validation accuracy: 0.3010


In [12]:
# ============================================================
# 6️⃣ Save Project Folder (✅ fixed for new Keras versions)
# ============================================================
import json, os, zipfile
from google.colab import files

project_dir = "AI_Content_Classifier"
os.makedirs(project_dir, exist_ok=True)

# ✅ Export the model in TensorFlow SavedModel format (best for Flask & Serving)
model.export(os.path.join(project_dir, "saved_model"))

# Save label encoder classes
with open(os.path.join(project_dir, "label_classes.json"), "w") as f:
    json.dump(list(le.classes_), f)

# ============================================================
# Flask API File
# ============================================================
serve_py = """import json
from flask import Flask, request, jsonify
import tensorflow as tf

app = Flask(__name__)

# ✅ Load model (compatible with .export format)
model = tf.keras.models.load_model('saved_model')
with open('label_classes.json') as f:
    classes = json.load(f)

@app.route('/')
def home():
    return {'message': 'AI_Content_Classifier API is running'}

@app.route('/predict', methods=['POST'])
def predict():
    data = request.get_json(force=True)
    texts = data.get('texts', [])
    if isinstance(texts, str):
        texts = [texts]
    preds = model.predict(texts)
    labels = [classes[int(i)] for i in preds.argmax(axis=1)]
    confs = [float(c) for c in preds.max(axis=1)]
    return jsonify({'predictions': [
        {'text': t, 'label': l, 'confidence': c}
        for t, l, c in zip(texts, labels, confs)
    ]})

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000)
"""
open(os.path.join(project_dir, "serve.py"), "w").write(serve_py)

# ============================================================
# Create requirements and Procfile
# ============================================================
open(os.path.join(project_dir, "requirements.txt"), "w").write(
    "tensorflow\nflask\ngunicorn\npandas\nscikit-learn\nmatplotlib\n"
)
open(os.path.join(project_dir, "Procfile"), "w").write(
    "web: gunicorn -w 2 -b 0.0.0.0:$PORT serve:app\n"
)



Saved artifact at 'AI_Content_Classifier/saved_model'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.string, name='keras_tensor')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  133413435276176: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133413435274640: TensorSpec(shape=(), dtype=tf.int64, name=None)
  133413435270416: TensorSpec(shape=(), dtype=tf.string, name=None)
  133413435268112: TensorSpec(shape=(), dtype=tf.int64, name=None)
  133413435273872: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133413435273680: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133413435272912: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133413435275024: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133413435274256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133413435274064: TensorSpec(shape=(), dtype=tf.resource, name=None)
  133413435275792:

46